In [1]:
# import data
import pandas as pd

bio = pd.read_excel("biomarkers.xlsx")
cov = pd.read_excel("covariates.xlsx", sheet_name='Ark1')

# Merge the data from both files based on the ID

In [2]:
# Extract only the values at the time of inclusion for bio
filtered_bio = bio[bio['Biomarker'].str.endswith('0weeks')]
# Count the number of rows
rows = filtered_bio.shape[0]
print(f'rows: {rows}')

rows: 117


In [3]:
# Extract the ID numbers from filtered_bio
filtered_bio['PatientID'] = filtered_bio['Biomarker'].apply(lambda x: x.split('-')[0])
print(filtered_bio)

      Biomarker  IL-8  VEGF-A    OPG  TGF-beta-1  IL-6  CXCL9  CXCL1  IL-18  \
0    126-0weeks  7.63   11.51  10.20        8.83  3.52   6.16   9.45   7.91   
2    127-0weeks  6.93   10.92  10.30        6.59  2.73   6.14   7.31   7.95   
5    128-0weeks  8.62   12.51  10.56        8.51  3.71   7.34   9.90   8.72   
8    129-0weeks  8.16   11.16  10.61        8.76  3.85   5.81   9.18   7.49   
11   130-0weeks  8.81   12.53  11.23        9.41  4.22   6.35   9.34   9.00   
..          ...   ...     ...    ...         ...   ...    ...    ...    ...   
336  118-0weeks  7.81   12.05  10.35        7.91  4.10   6.66   7.66   9.37   
338  119-0weeks  8.18   11.97  10.75        9.39  3.05   7.03   9.33   8.64   
341  121-0weeks  7.01   12.53  10.91        8.74  3.08   5.95   7.61   8.27   
344  122-0weeks  7.86   12.80  10.73        9.30  2.93   6.73   9.60   8.25   
346  124-0weeks  6.51   11.38  10.07        7.78  3.29   5.97   7.17   8.68   

     CSF-1 PatientID  
0     8.41       126  
2    

C:\Users\tanihara\AppData\Local\Temp\ipykernel_11788\1922404316.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_bio['PatientID'] = filtered_bio['Biomarker'].apply(lambda x: x.split('-')[0])


In [4]:
# Convert the ID columns in filtered_bio and cov to integer type
filtered_bio['PatientID'] = filtered_bio['PatientID'].astype(int)
cov['PatientID'] = cov['PatientID'].astype(int)

C:\Users\tanihara\AppData\Local\Temp\ipykernel_11788\1845910225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_bio['PatientID'] = filtered_bio['PatientID'].astype(int)


In [5]:
# Merge filtered_bio and cov based on the ID
merged_df = pd.merge(filtered_bio, cov, on='PatientID', how='inner')
print(merged_df)

      Biomarker  IL-8  VEGF-A    OPG  TGF-beta-1  IL-6  CXCL9  CXCL1  IL-18  \
0    126-0weeks  7.63   11.51  10.20        8.83  3.52   6.16   9.45   7.91   
1    127-0weeks  6.93   10.92  10.30        6.59  2.73   6.14   7.31   7.95   
2    128-0weeks  8.62   12.51  10.56        8.51  3.71   7.34   9.90   8.72   
3    129-0weeks  8.16   11.16  10.61        8.76  3.85   5.81   9.18   7.49   
4    130-0weeks  8.81   12.53  11.23        9.41  4.22   6.35   9.34   9.00   
..          ...   ...     ...    ...         ...   ...    ...    ...    ...   
112  118-0weeks  7.81   12.05  10.35        7.91  4.10   6.66   7.66   9.37   
113  119-0weeks  8.18   11.97  10.75        9.39  3.05   7.03   9.33   8.64   
114  121-0weeks  7.01   12.53  10.91        8.74  3.08   5.95   7.61   8.27   
115  122-0weeks  7.86   12.80  10.73        9.30  2.93   6.73   9.60   8.25   
116  124-0weeks  6.51   11.38  10.07        7.78  3.29   5.97   7.17   8.68   

     CSF-1  PatientID  Age  Sex (1=male, 2=female) 

# Building a Regression Model

In [6]:
# Remove rows containing missing values
cleaned_df = merged_df.dropna(subset=['Vas-12months'])

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

# Define dependent and independent variables
X = cleaned_df[['IL-8', 'VEGF-A', 'OPG', 'TGF-beta-1', 'IL-6', 'CXCL9', 'CXCL1', 'IL-18', 'CSF-1', 'Age', 'Sex (1=male, 2=female)', 'Smoker (1=yes, 2=no)']] 
y = cleaned_df['Vas-12months'] 

# Split the data into training and test sets (training:test = 80%:20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and fit a multiple regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Calculate the model's R^2 value for the training data
r2 = model.score(X_train, y_train)
# Calculate the adjusted coefficient of determination
n = X_train.shape[0]  # sample size
p = X_train.shape[1]  # the number of independent variables
adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
print(f'Adjusted R-squared: {adj_r2}')

# Display the fitted parameter values (coefficients) and the constant term (intercept)
print("Coefficient:")
coefficients = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
print(coefficients)
print("\nintercept:", model.intercept_)

Adjusted R-squared: 0.19205508975027463
Coefficient:
                        Coefficient
IL-8                       1.059788
VEGF-A                     1.473412
OPG                       -2.124257
TGF-beta-1                -1.021919
IL-6                       0.897379
CXCL9                     -0.312225
CXCL1                     -0.224792
IL-18                     -0.536127
CSF-1                      1.663469
Age                       -0.035122
Sex (1=male, 2=female)     0.253498
Smoker (1=yes, 2=no)      -1.124006

intercept: 3.287381412796533


In [15]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Calculate predicted values
y_pred = model.predict(X_train)

# Scatter plot and regression line
plt.figure(figsize=(8, 6))
plt.scatter(y_train, y_pred, color='blue', label='data point')
plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'k--', lw=2, label='regression line')
plt.xlabel('actual values', fontsize=20)
plt.ylabel('predicted values', fontsize=20)
plt.legend()
plt.savefig(f'plot_training.png')  
plt.close()  
    
# Residual plot
residuals = y_train - y_pred
plt.figure(figsize=(8, 6))
plt.scatter(y_pred, residuals, color='red', label='residuals')
plt.hlines(y=0, xmin=y_pred.min(), xmax=y_pred.max(), colors='black', linestyles='dashed')
plt.xlabel('predicted values', fontsize=20)
plt.ylabel('residuals', fontsize=20)
plt.legend()
plt.savefig(f'r-residuals.png')  
plt.close()  

# Histogram of residuals
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True, bins=20)
plt.xlabel('Residuals', fontsize=20)
plt.ylabel('Frequency', fontsize=20)
plt.savefig(f'r-residuals_hist.png')  
plt.close()  

In [18]:
# Calculate predicted values
y_pred = model.predict(X_test)

# Create a dataframe for comparing actual and predicted values
comparison_df = pd.DataFrame({'Actual Vas-12months': y_test, 'Predicted Vas-12months': y_pred})

# Display comparison results between actual and predicted values
comparison_df.reset_index(drop=True, inplace=True)
comparison_df

,Actual Vas-12months,Predicted Vas-12months
0,3.7,3.690403
1,5.0,6.108549
2,3.5,3.273627
3,0.0,3.238037
4,0.0,1.699745
5,9.5,5.217898
6,3.5,1.784642
7,9.0,2.658885
8,0.0,3.252753
9,0.0,3.591382


In [ ]:
# Scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(comparison_df['Actual Vas-12months'], comparison_df['Predicted Vas-12months'], color='blue', label='Data points')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2, label='Identity line')
plt.xlabel('Actual Vas-12months', fontsize=20)
plt.ylabel('Predicted Vas-12months', fontsize=20)
plt.legend()
plt.savefig(f'plot_test.png')  
plt.close() 

In [13]:
from sklearn.metrics import mean_squared_error

# Calculate Mean Squared Error (MSE) for statistical evaluation
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')

Mean Squared Error (MSE): 10.628931155806434


In [15]:
column_name = 'Vas-12months'

# Determine Mean, Variance, Maximum, and Minimum values
mean_value = cleaned_df[column_name].mean()
variance = cleaned_df[column_name].var()
max_value = cleaned_df[column_name].max()
min_value = cleaned_df[column_name].min()

(mean_value, variance, max_value, min_value)

(3.6339130434782607, 9.545594202898549, 10.0, 0.0)

# Reproduce regression analysis with statsmodels

In [15]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

X = cleaned_df[['IL-8', 'VEGF-A', 'OPG', 'TGF-beta-1', 'IL-6', 'CXCL9', 'CXCL1', 'IL-18', 'CSF-1', 'Age', 'Sex (1=male, 2=female)', 'Smoker (1=yes, 2=no)']]  
y = cleaned_df['Vas-12months']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = sm.add_constant(X_train)

model = sm.OLS(y_train, X_train).fit()

# 結果の要約を表示
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           Vas-12months   R-squared:                       0.299
Model:                            OLS   Adj. R-squared:                  0.192
Method:                 Least Squares   F-statistic:                     2.803
Date:                Sat, 10 Feb 2024   Prob (F-statistic):            0.00308
Time:                        21:12:30   Log-Likelihood:                -215.22
No. Observations:                  92   AIC:                             456.4
Df Residuals:                      79   BIC:                             489.2
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                      3